# Boosting Trees

In [ ]:
%matplotlib inline
from pylab import *
import pandas as pd
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split

## Gradient Boosting 

Commencons par ajuster un regresseur par gradient boosting sur des familles d'arbres.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

####  Préparation des données housing

In [ ]:
from sklearn.datasets import fetch_california_housing
california_housing = fetch_california_housing(as_frame=True)

In [ ]:
print(california_housing.DESCR)

In [ ]:
california_housing.data.head()

In [ ]:
X_housing = california_housing.data
Y_housing = california_housing.target
print(shape(X_housing))

Pour éviter que les temps de calcul soient trop longs, nous allons travailler avec un sous échantillon:

In [ ]:
from sklearn.utils import resample
X_housing, Y_housing  = resample(X_housing,Y_housing, n_samples = 2000, replace = False)

Découpage train / test :

In [ ]:
X_housing_train, X_housing_test, y_housing_train, y_housing_test = \
train_test_split(X_housing,Y_housing,test_size=0.5)

####  Fonctions Gradient Boosting de sckit-learn

Importation des fonctions Gradient Boosting de sckit-learn et de la fonction `mean_squared_error` pour le calcul des erreurs quadratique moyennes.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

Nous ajustons maintenant un modèle GBM avec les paramètres proposés ci-dessous.

> Retrouver la signification de chacun des paramètres.

In [ ]:
GBM = GradientBoostingRegressor(n_estimators=1000,
                                         max_depth=8,
                                         min_samples_split= 4,
                                         learning_rate=0.05,
                                         loss='squared_error')

print(GBM)

+ n_estimators : ###TO DO ### ?
+ learning_rate =  ###TO DO ###?
+ loss : ###TO DO ### ?
+ max_depth : ###TO DO ### ?
+ min_samples_split  : ###TO DO ### ?

Pour calculer l'erreur mse sur le test (aussi appelée deviance dans la doc) :

In [ ]:
GBM.fit(X_housing_train, y_housing_train)
mse = mean_squared_error(y_housing_test, GBM.predict(X_housing_test))
print(mse)

Attention : le score renvoyé ci-dessous est un $R^2$ i.e. variance expliquée par le prédicteur / variance totale de $Y$, voir [ici](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score-the-coefficient-of-determination).

In [ ]:
print(GBM.score(X_housing_test, y_housing_test))

Quelques principes à retenir:
+ Plus on ajuste d'arbres, plus le cout computationnel est élevé.
+ le nombre d'arbres correspond au nombre d'itérations.
+ Un taux d'apprentissage petit nécessitera plus d'arbres.
+ Un taux d'apprentissage trop élevé fera des sauts de gradients potentiellement trop grands, et au bout d'un certain nombre d'itérations il sera  difficile d'améliorer les scores.
+ Une bonne pratique consiste à choisir d'abord un taux d'apprentissage pas trop faible (pour ne pas faire exploser le nombre d'arbres) et à le diminuer ensuite, une fois ajustés les autres paramètres (nb de noeuds, profondeur ...).

#### Etude des erreurs le long des itérations -  learning rate

Nous allons maintenant étudier l'évolution de la perte en fonction du nombre d'itérations.

Noter que l'erreur d'apprentissage est accessible dans GBM:

In [ ]:
print(GBM.train_score_)

Nous pouvons aussi tracer l'évolution de l'erreur de test mse (ou bien le score $R^2$) le long des itérations. En effet, pour retrouver tous les estimateurs temporaires à chaque de l'étape de l'algo, il suffit de considérer la somme tronquée du prédicteur boosting final. 
> Utiliser les méthodes `staged_predict()` et `loss_()` de la classe `GradientBoostingRegressor` pour afficher l'erreur de test en fonction du nombre d'itérations. Comparer avec l'erreur sur le train. 


In [ ]:
test_score = []
for y_pred in  GBM.staged_predict(### TODO ###):
    test_score.append(### TODO ###)

In [ ]:
plt.plot(np.arange(GBM.n_estimators) + 1, ### TODO ###, 'b-',
         label='Training mse')
plt.plot(np.arange(GBM.n_estimators) + 1, ### TODO ###, 'r-',
         label='Test mse')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('mse');

> Est-il ici pertinent de faire du "early stopping" i.e. considérer une somme tronquée du prédicteur boosting pour contrôler le sur-apprentissage ?

In [ ]:
### TODO ###

> Superposer quelques trajectoires pour plusieurs valeurs du learning rate.

In [ ]:
### TODO ###

#### Choix des paramètres du GBM

> Pour `n_estimators=300` et `learning_rate=0.1`, effectuer une recherche avec par `GridSearch()` pour choisir `max_depth` et  `min_samples_split`. Faire cette recherche en "cross validant" uniquement l'échantillon d'apprentissage de façon à garder des données pour évaluer les performances du modèle finalement sélectionné.

In [ ]:
max_depth_values= [2,4,6,8] 
min_samples_split_values= [2,3,4]

import multiprocessing
multiprocessing.cpu_count()

In [ ]:
from sklearn.model_selection import GridSearchCV

GBM = ### TODO ###
param_grid = dict(m### TODO ###)
grid_search = GridSearchCV(### TODO ###)
grid_result = grid_search.fit(### TODO ###)

In [ ]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

> Une fois ces valeurs choisies, diminuer le learning rate et augmenter le nombre d'itérations pour essayer d'améliorer encore les performances.

In [ ]:
### TODO ###

> Calculer le $R^2$ et l'erreur de test pour le modèle final

In [ ]:
grid_result.best_estimator_.score(### TODO ###)

In [ ]:
mean_squared_error(### TODO ###)

#### Indices d'importance

> Il est aussi possible de calculer des indices d'importances pour les GMB avec la méthode `feature_importances_`. Comparer les importances du modèle GMB obtenu avec celles obtenues précédemment pour les forêts aléatoires.

In [ ]:
GBMbest = grid_result.best_estimator_

In [ ]:
# On commence ajuster le modele choisi sur toutes les données:
GBMbest.fit(X_housing,Y_housing);

In [ ]:
feature_importance = GBMbest.feature_importances_
### TODO : affichage ###

##  XGboost

Si nécessaire, installer la librairie [`xgboost`](https://xgboost.readthedocs.io/en/latest/install.html). 

In [ ]:
from xgboost import XGBRegressor

Nous allons utilser Xgboost avec pour classifieurs faibles des arbres (on pourait aussi utiliser aussi des modèles linéaires basés sur peu de variabes).

Les paramètres de Xgboost, notamment pour les fonctions `XGBRegressor()` et `XGBClassifier()`, sont décrits dans [cette page](http://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn). Elles sont similaires aux commdandes des fonctions de sckit-learn.

In [ ]:
XGBR = XGBRegressor()
XGBR.fit(X_housing_train, y_housing_train)
y_pred = XGBR.predict(X_housing_test)
XGBR

> Donner le score $R^2$ et l'erreur mse. Comparer avec GBM.

Les remarques précedentes sur les reglages des paramètres de GBM sont encore vraies pour Xgboost. Il faut en plus ajuster les paramètres `alpha`  et `lambda` des termes de régularisation.

> Choisir les paramètres de Xgboost.

D'abord avec un learning rate pas trop faible.

In [ ]:
max_depth= [2,4,6,8] 
min_child_weight = [2,4]
reg_alpha=  [0,0.1,1,2]
reg_lambda = [0,0.1,1,2]
### TODO ###

Et maintenant avec des learning rates plus petits:

In [ ]:
learning_rate = [0.001, 0.01,0.05]
n_estimators = [100,500,1000]

In [ ]:
### TODO ###

In [ ]:
### TODO : Graphe de la mse ###

 :

## Implémentation du Boosting Regressor 

> Implémenter votre propore fonction Tree Boosting Regressor, on considérant la perte $\ell_2$. On pourra pour cela, à chaque itération, 
ajuster un arbre de régression (de faible prodondeur) sur les résidus courants, à l'aide de la fonction `tree.DecisionTreeRegressor()` . L'étape 5 de l'Algorithme 6 "Gradient Tree Boosting Regressor Algorithm" donné en cours est-elle nécessaire dans ce contexte ? 


In [ ]:
### TODO ###